In [ ]:
!pip install langchain langgraph openai unstructured

In [ ]:
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
from dotenv import load_dotenv
import os

load_dotenv()

llm = ChatOpenAI(model="gpt-4o", temperature=0)
API_KEY = os.getenv("OPEN_AI_API_KEY")

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = API_KEY  # replace with your key

In [ ]:
from typing import TypedDict, Optional

class ResumeState(TypedDict):
    resume_text: Optional[str]
    job_text: Optional[str]
    job_requirements: Optional[str]
    tailoring_notes: Optional[str]
    tailored_resume: Optional[str]


In [ ]:
def parse_resume_node(state):
    return {"resume_text": state["resume_text"]}

def parse_job_node(state):
    return {"job_text": state["job_text"]}

In [ ]:
extract_requirements_node = RunnableLambda(
    lambda state: {
        "job_requirements": llm.invoke([
            HumanMessage(content=f"""
Given the job description below, extract the key requirements, including:
- Required skills
- Years of experience
- Tools/technologies
- Soft skills
- Any domain-specific expertise

Job Description:
{state['job_text']}
""")
        ]).content
    }
)

In [ ]:
match_skills_node = RunnableLambda(
    lambda state: {
        "tailoring_notes": llm.invoke([
            HumanMessage(content=f"""
You are comparing a résumé and a job description.

Résumé:
{state['resume_text']}

Job Requirements:
{state['job_requirements']}

Identify:
- What is already covered in the résumé
- What is missing
- Suggestions for how to tailor the résumé
Return actionable suggestions only.
""")
        ]).content
    }
)


In [ ]:
tailor_resume_node = RunnableLambda(
    lambda state: {
        "tailored_resume": llm.invoke([
            HumanMessage(content=f"""
Tailor the résumé below based on these tailoring notes to match the job requirements.

Résumé:
{state['resume_text']}

Tailoring Notes:
{state['tailoring_notes']}

Output a polished and tailored résumé. Keep formatting clean.
""")
        ]).content
    }
)


In [ ]:
graph_builder = StateGraph(ResumeState)

graph_builder.add_node("ParseResume", parse_resume_node)
graph_builder.add_node("ParseJob", parse_job_node)
graph_builder.add_node("ExtractRequirements", extract_requirements_node)
graph_builder.add_node("MatchSkills", match_skills_node)
graph_builder.add_node("TailorResume", tailor_resume_node)

# Edges
graph_builder.set_entry_point("ParseResume")
graph_builder.add_edge("ParseResume", "ParseJob")
graph_builder.add_edge("ParseJob", "ExtractRequirements")
graph_builder.add_edge("ExtractRequirements", "MatchSkills")
graph_builder.add_edge("MatchSkills", "TailorResume")
graph_builder.add_edge("TailorResume", END)

# Compile
graph = graph_builder.compile()


In [ ]:
inputs = {
    "resume_text": "<your raw résumé text here>",
    "job_text": "<your job description text here>"
}

result = graph.invoke(inputs)

print("\nTailored Résumé:\n")
print(result["tailored_resume"])
